# Bài tập 2

Nguyễn Trọng Nghĩa - 1712615

## Câu 1 (1 điểm)

In [1]:
import numpy as np
head = np.random.binomial(10,0.5,(100000,1000)) # tung một 1000 đồng xu với xác suất 0,5 ,thực hiện lặp 100000 lần
print("Vmin trung bình với 100000 lần thực hiện:",np.mean(np.min(np.divide(head,10),axis=1)))

Vmin trung bình với 100000 lần thực hiện: 0.037694


`Vmin` $\approx$ `0.04` nên chọn giá trị gần nhất là `0.01`

Đáp án là: [b]

## Câu 2 (1 điểm)

In [2]:
import math
head = np.random.binomial(10,0.5,(100000,1000))/10  # mô phỏng tung 1000 đồng xu, thực hiện 100000 lần
headc1 =np.absolute(head[0]-0.5)   # đồng xu thứ nhất head[0] từ 1000 đồng xu
headcrand = np.absolute(head-0.5)  # đồng xu random trong 1000 đồng xu, mỗi lần thực hiện sẽ random các đồng xu khác nhau
headcrand = headcrand[np.arange(headcrand.shape[0]),np.random.choice(np.arange(headcrand.shape[1]), size=headcrand.shape[0])]
headcmin = np.absolute(np.min(head,axis=1)-0.5)  # đồng xu có mặt nhỏ nhất trong 1000 đồng xu
# head[head>0]
def ishoeffding(head,N): # hàm kiểm ra Hoeffding 
    for i in np.arange(0.01,0.5,0.01):  # với mỗi epsilon (0< epsilon <0.5) nếu VT > VT thì không theo BDT Hoeffding
        if(len(np.where(head>i)[0])/100000) > 2*math.exp(-2*(np.around(i,2)**2)*N):  
            return False
    return True
print("c1 satisfies the Hoeffding's inequality: ",ishoeffding(headc1,10))  # xuất kết quả của mỗi đồng xu
print("cmin satisfies the Hoeffding's inequality: ",ishoeffding(headcmin,10))
print("crand satisfies the Hoeffding's inequality: ",ishoeffding(headcrand,10))

c1 satisfies the Hoeffding's inequality:  True
cmin satisfies the Hoeffding's inequality:  False
crand satisfies the Hoeffding's inequality:  True


Dựa vào các kết quả trên : `c1 và crand` thỏa mãn bất đẳng thức `Hoeffding's inequality`

Đáp án là: [d]

## Câu 3 (1 điểm)

`Hypothesis h` có độ lỗi so với `hàm mục tiêu f` là $\mu$ (tức là $y \neq f(x)$)

Noise của f được xác định:
$$ P(y|x) \left\{\begin{matrix}
\lambda & y = f(x) \\ 1-\lambda & y \neq f(x)
\end{matrix}\right.$$

$P(y|x)$ tạo thành một hệ biến cố đầy đủ: $P(h \approx y) = $ Độ chính xác `h` với nhiễu + Độ lỗi `h` với dữ liệu chuẩn $$=P(f(x) _{}\neq y) * P(h = f(x)) +P(f(x) =y)* P(h _{}\neq f(x)) =(1-\lambda)*(1-\mu) + \lambda*\mu$$

Đáp án là: [e]

## Câu 4 (1 điểm)

Ta có: $P(h \approx y) = (1-\lambda)*(1-\mu) + \lambda*\mu = 1 + 2 *\lambda * \mu - \mu - \lambda$

Để độ hiệu quả của `h` độc lập (tức là không phụ thuộc) với $\mu$ thì $\Rightarrow 2 *\lambda * \mu - \mu =0 \Rightarrow\lambda = \frac{1}{2}$

Đáp án là: [b]

## Câu 5 (1 điểm)

In [3]:
def generate_target_w(): # khởi tạo hàm mục tiêu với đường thẳng nối 2 điểm random theo phân phối đều [-1, 1]x[-1, 1]
    """ 
    Generates target_w (the vector of parameters of f) 
    from two random, uniformly distributed points in [-1, 1] x [-1, 1].
    
    Returns
    -------
    target_w : numpy array, shape (3, 1) 
        The vector of parameters of f.
    """
    # Generate two points from a uniform distribution over [-1, 1]x[-1, 1]
    p1 = np.random.uniform(-1, 1, 2)
    p2 = np.random.uniform(-1, 1, 2)
    # Compute the target W from these two points
    target_w = np.array([p1[1]*p2[0] - p1[0]*p2[1], p2[1] - p1[1], p1[0] - p2[0]]).reshape((-1, 1)) 
    
    return target_w

In [4]:
def generate_data(N, target_w): # phat sinh du lieu mau tranh du lieu nam tren duong thang ham muc tieu f
    """
    Generates a data set by generating random inputs and then using target_w to generate the 
    corresponding outputs.
    
    Parameters
    ----------
    N : int
        The number of examples.
    target_w : numpy array, shape (3, 1) 
        The vector of parameters of f.
    
    Returns
    -------
    X : numpy array, shape (N, 3)
        The matrix of input vectors (each row corresponds to an input vector); the first column of 
        this matrix is all ones.
    Y : numpy array, shape (N, 1)
        The vector of outputs.        
    """
    bad_data = True # `bad_data = True` means: data contain points on the target line 
                    # (this rarely happens, but just to be careful)
                    # -> y's of these points = 0 (with np.sign); 
                    #    we don't want this (y's of data must be -1 or 1)
                    # -> re-generate data until `bad_data = False`
    
    while bad_data == True:
        X = np.random.uniform(-1, 1, (N, 2))
        X = np.hstack((np.ones((N, 1)), X)) # Add 'ones' column
        Y = np.sign(np.dot(X, target_w))
        if (0 not in Y): # Good data
            bad_data = False
    
    return X, Y

In [5]:
def linearRegression(X, Y): #tìm đường hồi quy tuyến tính
    """
    Parameters
    ----------
    X : numpy array, shape (N, 3)
        The matrix of input vectors (each row corresponds to an input vector); the first column of 
        this matrix is all ones.
    Y : numpy array, shape (N, 1)
        The vector of outputs.
    
    Returns
    -------
    w : numpy array, shape (3, 1) 
        The vector of parameters of g.
    """
    X_dagger = np.dot(np.linalg.inv(np.dot(X.T, X)), X.T) # cực tiểu độ lỗi MSE
    w = np.dot(X_dagger, Y)
        
    return w

In [6]:
def avgerr(N):  # độ lỗi trung bình trên tập train với linear Regression
    """
    Parameters
    ----------
    N : int
        The number of training examples.
    """
    num_runs = 1000
    avg_train_err = 0.0 # giá trị Độ lỗi trung bình trên tập train Ein
    avg_test_err = 0.0 # giá trị nhận độ lỗi trung bình trên test Eout
    
    for r in range(num_runs):
        # Generate target_w (Khởi tạo ngẫu nhiên hàm mục tiêu f(x))
        target_w = generate_target_w()
        
        # Generate training set (tạo dữ liệu train trên f(x))
        X, Y = generate_data(N, target_w)
        
        #tìm g dựa trên cực tiểu hóa đạo hàm
        w = linearRegression(X, Y)

        # Test g trên tập train (độ lỗi trên dữ liệu train)
        train_err = np.mean(np.sign(np.dot(X, w)) != Y)
         # Phát sinh 1000 dữ liệu test
        X_test, Y_test = generate_data(1000, target_w)
        
        # Test g trên tập test (độ lỗi trên dữ liệu test)
        test_err = np.mean(np.sign(np.dot(X_test, w)) != Y_test)
        
        # Cập nhật độ lỗi trung bình Ein và Eout
        avg_test_err += (test_err * 1.0 / num_runs)
        avg_train_err += (train_err * 1.0 / num_runs)
    
    # xuất kết quả
    print('avg_train_err = %f' % (avg_train_err))
    print('avg_test_err = %f' % (avg_test_err))

In [7]:
avgerr(100)

avg_train_err = 0.040500
avg_test_err = 0.048887


Kết quả (độ lỗi trung bình Ein) `avg_train_err` $\approx$ 0.03 nên chọn đáp án gần nhất là: `0.01`

Đáp án là: [c]

## Câu 6 (1 điểm)

Kết quả từ câu 9 (độ lỗi trung bình Eout) `avg_test_err` $\approx$ 0.04 nên chọn đáp án gần nhất là: `0.01`

Đáp án là: [c]

## Câu 7 (1 điểm)

In [8]:
def PLA(X,Y): # perceptron learning algorithm
    
    #khởi tạo trọng số w bằng linearRegession
    w = np.dot(np.dot(np.linalg.inv(np.dot(X.T, X)), X.T),Y) # dùng trọng số của linear regression bất đầu cho PLA
    
    iteration = 0  # số lần cập nhật để phân tách tốt (số lần để hội tụ)
    
    w_bad = True;
    while w_bad:
        miss = np.where(np.sign(np.dot(X,w))!=Y)[0] # lấy mảng vị trí những điểm bị phân lớp sai
        if (miss.size == 0):  # nếu mảng rỗng thì phân tách thành công, kết thúc PLA
            break;
        random = np.random.choice(miss)  # chọn random trong những điểm phân lớp
        w = w + Y[random]*X[random].reshape(-1,1) # cập nhật giá trị w cho điểm lỗi
        iteration +=1 # tăng số lần hội tụ (số lần thực thi cập nhật) lên 1
        
    return w, iteration

In [9]:
def main(N): # tính số lần để hội tụ 
    
    num_runs =1000 # thực hiện trung bình 1000 lần
    avg_num_iterations = 0.0 # khởi tạo số trung bình số lần hội tụ (phân tách hoàn toàn)
    for r in range(num_runs):
        # Generate target_w
        target_w = generate_target_w()
        
        # Generate training set
        X, Y = generate_data(N, target_w)
        
        # tìm w với PLA  với w khởi tạo bằng linear Regression, và số lần để hội tụ
        w, num_iterations = PLA(X, Y)
        
        # Update average values
        avg_num_iterations += (num_iterations * 1.0 / num_runs)
        
    # Print results
    print('avg_num_iterations = %f' % (avg_num_iterations))

In [10]:
main(10)

avg_num_iterations = 3.434000


Kết quả `avg_num_iterations` $\approx$ 4 nên chọn đáp án gần nhất là: `1`

Đáp án là: [a]

## Câu 8 (1 điểm)

In [11]:
def computeY(X): # tính giá trị Y của các bộ dữ liệu X theo hàm mục tiêu
    Y=[]
    for i in X[0:,:]:
        results = np.sign(i[1]**2 + i[2]**2 -i[0]*0.6) # hàm mục tiêu f (x1**2 +x2**2-0.6)
        Y.append(results)
    return np.reshape(Y, (-1, 1)) # kích thước (N,1)

In [12]:
def generate_Data(N): # phát sinh dữ liệu ngẫu nhiên theo phân phối đều 
    data_bad = True
    while data_bad:
        X = np.random.uniform(-1,1,(N,2))
        X = np.hstack((np.ones((N,1)),X)) # thêm cột giá trị w0=1
        Y = computeY(X) # Y shape(N,1)
        if (0 not in Y):  # tránh dữ liệu nằm trên hàm mục tiêu
            data_bad = False
    return X,Y  

In [13]:
def generate_Noise(Y,N): # phát sinh dữ liệu nhiễu
    random = np.random.choice(np.arange(N), size=int(0.1*N), replace=False) # lấy random 10% số lượng mẫu 
    for i in random: # đổi ngược Y tại những vị trí random được
        if (Y[i]==1):
            Y[i] = -1
        elif(Y[i] == -1):
            Y[i] = 1 
    return Y     # shape(N,1)     

In [14]:
def EinNonlinear(N): # độ lỗi trên tập train với Linear Regression với 10% dữ liệu nhiễu
    num_runs =1000
    avg_train_err =0.0
    for r in range(num_runs):
        X,Y = generate_Data(N) # phát sinh tập dữ liệu
        Y = generate_Noise(Y,N)   # tạo nhiễu cho Y
        w = linearRegression(X,Y) # tìm g theo linear regression
        test_err = np.mean(np.sign(np.dot(X, w)) != Y)  # tính trung bình độ lỗi
        
        # Update average values
        avg_train_err += (test_err * 1.0 / num_runs)
    
    # Print results
    print('avg_train_err = %f' % (avg_train_err))

In [15]:
EinNonlinear(1000)

avg_train_err = 0.504049


Kết quả `avg_train_err` $\approx$ 0.5 nên chọn đáp án gần nhất là: `0.5`

Đáp án là: [d]

## Câu 9 (1 điểm)

In [16]:
def transformPoly(X):
    x1x2 = np.multiply(X[:,1],X[:,2]) # tạo dữ liệu cột x1*x2
    x1x1 = np.multiply(X[:,1],X[:,1])  #tạo dữ liệu cột x1**2
    x2x2 = np.multiply(X[:,2],X[:,2])  #tạo dữ liệu cột x2**2
    X = np.column_stack((X,x1x2,x1x1,x2x2))  # thêm dữ liệu 3 cột vào numpy array X
    return X

In [17]:
def EoutNonlinear(N):  # Độ lỗi trung bình trên tập test (có sử dụng lại một số hàm trên câu 8,5)
    num_runs =1000
    avg_err =0.0
    avg_w = np.zeros((6,1)) # khởi tạo trọng số w = 0
    for r in range(num_runs):
        X,Y = generate_Data(N)  # phát sinh dữ liệu theo f (x1**2 +x2**2-0.6)
        Y = generate_Noise(Y,N)  # phat sinh dữ liệu noise
        X_poly = transformPoly(X) # chuyển đổi sang đa chiều (2 chiều -> 5 chiều)
        w = linearRegression(X_poly,Y) # tìm w theo linear Regression
        
        X_test,Y_test = generate_Data(1000) # phát sinh 1000 dữ liệu mới
        Y_test = generate_Noise(Y_test,1000) # phát sinh noise cho dữ liệu mới
        X_poly_test = transformPoly(X_test)   # chuyển đổi dữ liệu sang đa chiều
        test_err = np.mean(np.sign(np.dot(X_poly_test, w)) != Y_test)  # kiểm tra độ chính xác trên tập test
        avg_err += (test_err * 1.0 / num_runs)  # cộng dồn cập nhật giá trị trung bình qua 1000 lần chạy
        avg_w += (w*1.0/num_runs) # tính trung bình w tìm được
    
    print('avg_test_err = %f' % (avg_err)) # xuất kết quả trung bình độ lỗi trên tập test qua 1000 lần chạy
    print('avg_w =',np.around(avg_w.T,3)) # xuất w tìm được

In [18]:
EoutNonlinear(1000)

avg_test_err = 0.126202
avg_w = [[-0.992  0.003  0.     0.003  1.558  1.559]]


`avg_w` có $w_{0} \approx -1 , w_{1} \approx 0.005,  w_{2} \approx 0.003, w_{3} \approx -0.002, w_{4} \approx 1.5$ và $w_{5} \approx 1.5$

`Hypothesis g` gần nhất, có nhiều $w$ tương đồng là [a]  có $w_{0} = -1, w_{4} = 1.5$ và $w_{5} = 1.5$ 

Đáp án là: [a]

## Câu 10 (1 điểm)

Kết quả từ câu 9 `avg_test_err` $\approx$ 0.13 nên chọn đáp án gần nhất là: `0.1`

Đáp án là: [b]